In [1]:
import torch
import os
import sys
#add sys so we can import from parent directory
sys.path.append('./model')
from model.dit import DiT_models
from model.vae_new import VAE_models
from torchvision.io import read_video, write_video
from utils import load_prompt, load_actions, sigmoid_beta_schedule
from tqdm import tqdm
from einops import rearrange
from torch import autocast
from safetensors.torch import load_model
import argparse
from pprint import pprint
import os

assert torch.cuda.is_available()
device = "cuda:0"



/home/kangrui/miniconda3/envs/oasis/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

# load DiT checkpoint
# model = DiT_models["DiT-S/2"](input_height=360,
#         input_width=640,)
vae = VAE_models["vit-l-20-shallow-encoder"](input_height=360,
        input_width=640,)

In [3]:
vae = vae.to(device).eval()
model=model.to(device).eval()

NameError: name 'model' is not defined

In [4]:
x = load_prompt(
        "/home/kangrui/projects/world_model/open-oasis/sample_data/sample_image_0.png",
        video_offset=0,
        n_prompt_frames=1,
    )

prompt is image; ignoring video_offset and n_prompt_frames


In [5]:
x = x.to(device)
   
    # vae encoding
B = x.shape[0]
H, W = x.shape[-2:]
scaling_factor = 0.07843137255
x = rearrange(x, "b t c h w -> (b t) c h w")
with torch.no_grad():
    with autocast("cuda", dtype=torch.half):
        x = vae.encode(x * 2 - 1).mean * scaling_factor
x = rearrange(x, "(b t) (h w) c -> b t c h w", t=1, h=H // vae.patch_size, w=W // vae.patch_size)
x = x[:, :1]
print(x.shape)

torch.Size([1, 1, 16, 18, 32])
